In [1]:
import sys
import os
import env_var
import spark_session
import validate
import logging
import logging.config
import ingest
import data_prep

In [2]:
%time

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs


In [3]:
open('application.log', 'w').close() ## To reset the application.log for every notebook run

In [4]:
logging.config.fileConfig('logging.config')
logger = logging.getLogger('root')
logger.setLevel(logging.DEBUG)

In [5]:
%time

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


In [6]:
spark = spark_session.create_spark_session()

2025-02-21 19:15:12,941 - root - INFO - Creating Spark Session... create_spark_session Method started
2025-02-21 19:15:12,942 - root - INFO - Calling spark object...


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/21 19:15:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


2025-02-21 19:15:17,496 - root - INFO - Validating the Spark object


In [7]:
%time

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.48 µs


In [8]:
file_format, header, inferschema, path_file = ingest.get_src_path(env_var.src_path,'csv')

2025-02-21 19:15:20,631 - root - WARNING - Retrieving the file name from the path -> /home/jupyteruser/work/data
2025-02-21 19:15:20,633 - root - WARNING - Successfully retrieved the file and path -> /home/jupyteruser/work/data/btc_15m_data_2018_to_2025.csv
2025-02-21 19:15:20,634 - root - WARNING - Successfully retrieved the file and path -> /home/jupyteruser/work/data/btc_1h_data_2018_to_2025.csv
2025-02-21 19:15:20,634 - root - WARNING - Path information has been read successfully of csv


In [9]:
%time

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.48 µs


In [10]:
main_df = ingest.ingest_data(in_spark=spark,in_file_path=path_file,in_file_format=file_format,in_header=header,in_schema=inferschema)

In [11]:
%time

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.44 µs


In [12]:
main_df = data_prep.renm_nd_chg_typ(main_df, env_var.new_cols)

if __name__ == '__main__':
    # creates spark session/object
    spark = spark_session.create_spark_session()
    logging.info('Application done..')
    # Reads the files from source path and assigns base parameters using the file_type
    file_format, header, inferschema, path_file = ingest.get_src_path(env_var.src_path,'csv')
    # Creates a dataframe using the ingest_data method of ingest.py file
    main_df = ingest.ingest_data(in_spark=spark,in_file_path=path_file,in_file_format=file_format,in_header=header,in_schema=inferschema)
    # Renaming the columns and changing datatypes
    main_df = data_prep.renm_nd_chg_typ(main_df, env_var.new_cols)
    # Stops the spark session/object
    spark_session.stop_spark_session(spark)

In [13]:
main_df.describe()

DataFrame[summary: string, open: string, high: string, low: string, close: string, Volume: string, Quote asset volume: string, Number of trades: string, Taker buy base asset volume: string, Taker buy quote asset volume: string, ignore: string, vol: string, quote_asset_vol: string, no_of_trades: string, taker_buy_base_asset_vol: string, taker_buy_quote_asset_vol: string]

In [15]:
from pyspark.sql.functions import *

In [18]:
missing_values = main_df.select([count(when(col(c).isNull(), c)).alias(c) for c in main_df.columns])
missing_values.show()

+---------+----+----+---+-----+------+----------+------------------+----------------+---------------------------+----------------------------+------+-------+---+--------+---------------+------------+------------------------+-------------------------+
|Open time|open|high|low|close|Volume|Close time|Quote asset volume|Number of trades|Taker buy base asset volume|Taker buy quote asset volume|ignore|open_tm|vol|close_tm|quote_asset_vol|no_of_trades|taker_buy_base_asset_vol|taker_buy_quote_asset_vol|
+---------+----+----+---+-----+------+----------+------------------+----------------+---------------------------+----------------------------+------+-------+---+--------+---------------+------------+------------------------+-------------------------+
|        0|   0|   0|  0|    0|     0|         0|                 0|               0|                          0|                           0|     0|      0|  0|       0|              0|           0|                       0|                       

In [19]:
# Check for trends using rolling average
from pyspark.sql.window import Window
window_spec = Window.orderBy("open_tm").rowsBetween(-30, 30)
main_df = main_df.withColumn("rolling_avg", mean(col("close")).over(window_spec))

In [ ]:
main_df.show()

In [21]:
# Remove seasonality (assuming weekly seasonality)
df = main_df.withColumn("week", weekofyear("open_tm"))
seasonal_mean = df.groupBy("week").agg(mean(col("close")).alias("seasonal_mean"))
df = df.join(seasonal_mean, on="week", how="left").withColumn("deseasonalized_value", col("close") - col("seasonal_mean"))

In [23]:
# Check for stationarity using Augmented Dickey-Fuller (ADF) test
from statsmodels.tsa.stattools import adfuller
adf_test = adfuller(df.select("deseasonalized_value").rdd.flatMap(lambda x: x).collect())
print(f"ADF Test Statistic: {adf_test[0]}")
print(f"p-value: {adf_test[1]}")

ADF Test Statistic: 0.0963903148234034
p-value: 0.9658137782553056


In [24]:
print(f"p-value: {adf_test}")

p-value: (0.0963903148234034, 0.9658137782553056, 49, 62277, {'1%': -3.4304550077806923, '5%': -2.861586411484126, '10%': -2.566794703262921}, 872129.540202223)
